# DEPENDENCIES

In [1]:
%config InlineBackend.figure_format ='retina'
import pandas as pd
import os

In [2]:
# To get all results printed: (or "last_expr" to only last expression from cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)  # To show all columns on results
pd.set_option('display.max_rows', None)     # To show all rows on results
pd.set_option('display.max_colwidth', -1) # To display all length of each cell

<ipython-input-2-4615a404387f>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1) # To display all length of each cell


# WORKING DIRECTORY

In [3]:
%cd ..
main_path = os.getcwd() + "/"
print("main_path is: ",main_path)

/Users/mlcanales/Documents/(Git) Projects/Research Quilting
main_path is:  /Users/mlcanales/Documents/(Git) Projects/Research Quilting/


# BEGIN

https://www.bibliometrix.org/documents/Field_Tags_bibliometrix.pdf

Run `add_country` and `add_CR_AU` before changing the labels with `label_remove` and `relabel`.

In [425]:
df = pd.read_csv("scopus.csv")

In [417]:
def label_remove(df):
    """
    Takes a scopus csv transformed into dataframe, and removes labels not needed.
    """
    df.drop(columns = ['Author(s) ID', 'Art. No.', 'Link', 'Affiliations',
                       'Molecular Sequence Numbers', 'Chemicals/CAS', 'Tradenames',
                       'Manufacturers', 'Funding Details', 'Funding Text 1',
                       'Correspondence Address', 'Editors', 'Sponsors','Conference code',
                       'CODEN', 'PubMed ID','Publication Stage', 'Access Type', 'EID'], inplace = True)
    return()

In [418]:
def relabel(df):
    """
    Takes a scopus csv transformed into dataframe, and renames labels to match Bibliometrix's.
    """
    df.rename(columns={
        'Authors':'AU', 
        'Title':'TI', 
        'Source title':'SO', 'Abbreviated Source Title':'JI',
        'Source':'DB',
        'Document Type':'DT',
        'Year':'PY',
        'Author Keywords':'DE', 'Index Keywords':'ID', 
        'Abstract':'AB', 
        'Authors with affiliations':'C1',
        'Cited by':'TC', 
        'References':'CR', 
        'Volume':'VL', 
        'Issue':'IS',
        'Language of Original Document':'LA',
        'Conference name': 'CT', 'Conference date':'CY', 'Conference location':'CL', 
        'ISSN':'SN', 'ISBN':'BN', 'DOI':'DI', 
        'Page start':'BP', 'Page end':'EP', 'Page count':'PG',
        'Publisher':'PU'}, 
              inplace = True)
    return()

In [419]:
def add_country(df):
    """
    Reads the "Affiliations" column and: (1) extracts the country of the first author, adding 
    it to df in column "AU1_CO"; (2) extracts the country of the co-authors, adding it to df
    in column "AU_CO".
    """
    df["AU1_CO"] = ""
    df["AU_CO"] = ""
    df['AU1_CO'] = df['AU1_CO'].astype(object)
    df['AU_CO'] = df['AU_CO'].astype(object)
    
    AU_CO_list = []   # Co-author's countries list
    
    for i in range(0,df.shape[0]):          # Iterate over all rows
        element = df.loc[i,'Affiliations']  # Select the cell
        
        if pd.isna(element) == True:  # If cell = nan
            df.at[i, 'AU_CO'] = 'nan'
            df.at[i, 'AU1_CO'] = 'nan'
            
        elif pd.isna(element) == False:  # cell != nan
            for j in range(0,len(element.split(';'))):
                if j == 0:  # First Author
                    df.at[i, 'AU1_CO'] = df.loc[i,'Affiliations'].split(';')[j].split(',')[-1]
                    
                else:       # Co-authors
                    AU_CO_list.append(df.loc[i,'Affiliations'].split(';')[j].split(',')[-1])
                
            df.at[i, 'AU_CO'] = ';'.join(AU_CO_list)
            AU_CO_list = []
        else:
            'Something weird happened'
    return()

In [420]:
def add_CR_AU(df):
    """
    Creates the "CR_AU" column based on the "References" column of the dataframe. 
    """
    
    df["CR_AU"] = ""
    df['CR_AU'] = df['CR_AU'].astype(object)
    lista = []
    for row in range(0, df.shape[0]):  # Iterate over all rows
        
        references = df.loc[row,'References'] # All references of each row
        if pd.isna(references) == True:  # If cell = nan
            df.at[row, 'CR_AU'] = 'nan'
            
        elif pd.isna(references) == False:  # cell != nan
            for i in range(0, len(references.split(';'))):  # Get first author
                lista.append(references.split(';')[i].split(',')[0])
            df.at[row, 'CR_AU'] = ';'.join(lista)
            lista = []
        else:
            'Something weird happened'

In [422]:
add_country(df)
add_CR_AU(df)
relabel(df)
label_remove(df)
df.to_csv("new_scopus.csv", index = False)

()

()

()

In [431]:
from pybliometrics.scopus import AbstractRetrieval

In [432]:
ab = AbstractRetrieval("https://www.scopus.com/inward/record.uri?eid=2-s2.0-0037076429&doi=10.1073%2fpnas.082081099&partnerID=40&md5=6c886d82996e9f30136130182130265f")
ab.title

Scopus401Error: The requestor is not authorized to access the requested view or fields of the resource

In [ ]:

>>> ab.publicationName
'SoftwareX'
>>> ab.authors
[Author(auid='57209617104', indexed_name='Rose M.E.', surname='Rose',
 given_name='Michael E.', affiliation=['60105007']),
 Author(auid='7004212771', indexed_name='Kitchin J.R.', surname='Kitchin',
 given_name='John R.', affiliation=['60027950'])]
>>>
>>> # Author-specific information
>>> from pybliometrics.scopus import AuthorRetrieval
>>> au2 = AuthorRetrieval(ab.authors[1].auid)
>>> au2.h_index
'30'
>>> au1 = AuthorRetrieval(ab.authors[0].auid)
>>> au1.affiliation_current
[Affiliation(id='60105007', parent=None, type='parent', relationship='author',
 afdispname=None, preferred_name='Max Planck Institute for Innovation and Competition',
 parent_preferred_name=None, country_code='deu', country='Germany',
 address_part='Marstallplatz 1', city='Munich', state=None,
 postal_code='80539', org_domain=None, org_URL='http://www.ip.mpg.de/')]
>>>
>>> # Affiliation information
>>> from pybliometrics.scopus import ContentAffiliationRetrieval
>>> aff1 = ContentAffiliationRetrieval(au1.affiliation_current[0].id)
>>> aff1.author_count
'72'